In [21]:
from generate_psf import fit_gaussian, weighted_linear_fit
import numpy as np
import cupy as cp
import os
import glob
import skimage
import scipy
from skimage.filters.rank import threshold
from tqdm.auto import tqdm
import cv2
from i_o import Paths
import json
from daio.h5 import lazyh5
import importlib
import h5py
cmap = "gray"

In [ ]:
# pn_psf = r"Y:/hpc/r/lfm2025/psfs"
pn_psfs = r"~/hpc-rw/lfm/psfs"
includes = ['']
excludes = []

recs = [os.path.basename(rec) for rec in sorted(glob.glob(os.path.expanduser(os.path.join(pn_psfs, '*'))), reverse=True) if os.path.exists(os.path.join(pn_psfs, rec, 'psf.h5'))]
recs = [rec for rec in recs if all([s in rec for s in includes]) and not any([s in rec for s in excludes])]
print('\n'.join(recs))

In [ ]:
# pn_rec = r"Y:/hpc/r/lfm2025/recordings"
pn_rec = r"~/hpc-r/lfm2025/recordings"
includes = ['']
excludes = []

recs = [os.path.basename(rec) for rec in sorted(glob.glob(os.path.expanduser(os.path.join(pn_rec, '*'))), reverse=True) if os.path.exists(os.path.join(pn_rec, rec, 'data.h5'))]
recs = [rec for rec in recs if all([s in rec for s in includes]) and not any([s in rec for s in excludes])]
print('\n'.join(recs))

In [ ]:
pn_bg = r"~/hpc-rw/lfm/bg"
for file in pathlib.Path(os.path.expanduser(pn_bg)).iterdir():
    if file.is_file():
        print(file.name)

In [5]:
pn_psfs = r"~/hpc-rw/lfm/psfs"
pn_rec = r"~/hpc-r/lfm2025/recordings"
pn_bg = r"~/hpc-rw/lfm/bg"
url_home = r"/home/lubo12/"
pn_out = r"~/hpc-rw/lfm/results"

dataset_name = "20250701_1605_LB_ZF_v552_f6_40fps_1"
psf_name = "20250701_1216_PSF_LB_noap_1"
bg_name = "20250701_1651_LB_bg_40fps.npy"

paths = Paths(dataset_name=dataset_name,
              psf_name = psf_name,
              bg_name=bg_name,
              pn_bg= pn_bg,
              pn_rec = pn_rec,
              pn_psfs=pn_psfs,
              pn_out=pn_out,
              url_home=url_home,
              )

In [ ]:
import video
importlib.reload(video)
from video import create_projection_image, recording_to_video, img_to_overlay_preview, recording_to_overlay_preview

In [10]:
import video
importlib.reload(video)
from video import recording_to_video
video_fn = recording_to_video(paths, fps=40, vmax=100, absolute_limits=False)

Creating video:   0%|          | 0/4020 [00:00<?, ?it/s]

Exception ignored in: <function AVWriter.__del__ at 0x14dc968aed40>
Traceback (most recent call last):
  File "/home/lubo12/LFM/imaging/video.py", line 59, in __del__
    self.close()
  File "/home/lubo12/LFM/imaging/video.py", line 53, in close
    for packet in self.stream.encode():
                  ^^^^^^^^^^^^^^^^^^^^
  File "av/video/stream.pyx", line 23, in av.video.stream.VideoStream.encode
  File "av/video/stream.pyx", line 32, in av.video.stream.VideoStream.encode
  File "av/codec/context.pyx", line 399, in av.codec.context.CodecContext.encode
  File "av/codec/context.pyx", line 319, in _send_frame_and_recv
  File "av/error.pyx", line 428, in av.error.err_check
av.error.EOFError: [Errno 541478725] End of file
